In [15]:
import re
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline

from sentence_transformers import SentenceTransformer

In [16]:

classifier = pipeline("text-classification", model="Falconsai/intent_classification")
text = "set tempo to 90 bpm"
result = classifier(text)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [17]:
result

[{'label': 'appointment', 'score': 1.0}]

In [ ]:
from sentence_transformers import SentenceTransformer

# intents = ["Play music", "Pause music", "Stop music"]
# intent_embeddings = model.encode(intents)

NameError: name 'model' is not defined

### Embeddings for the intentions

In [ ]:
# intent_embeddings

array([[ 0.6888892 , -0.4553661 ,  0.33920723, ...,  0.30810612,
         0.21911076,  0.3688749 ],
       [ 0.38095832, -0.3975439 ,  0.06559265, ...,  0.19594586,
        -0.27330932,  0.3508724 ],
       [ 0.4694603 , -0.6657139 ,  0.28274104, ...,  0.25762236,
        -0.01942625,  0.4449004 ]], dtype=float32)

##### Commands

In [ ]:
# b : true/flase
# i : number
# f : number
# s : text
# commands: 
cmds_raw = """AbletonLinkEnabled b
AbletonLinkLatency f
AddRelatedWorld s
AllowControlPanel b
AllowEditing b
AllowForking b
AllowJoltPatching b
AllowOpenBubbles b
AllowPlayerScaling b
AllowPlayerTilt b
AllowRadialMenu b
AllowSwimming b
AmbientColorHSV f f f
AmbientColorRGB f f f
AssignAccessLevel i
Autosaving b
BGColor1 f f f
BGColor2 f f f
BgColorHSV f f f
CallEvent i f
ChangeFogColor c
ChangeFogDistance f
CheckChangeRate f
CurrentStateToTemp
DisableGhostHitInteractions b
DisableGhostInteractions b
DisplayButtonText b
EditModeNumbers b
EnableBeta b
FadeToBlack f
FogColorHSV f f f
FogColorRGB f f f
FogFalloffMode i
GetCpuUsage
Go s
GoToTemp
HideAll
HighlightControllerButton s s f f
LoadFromCloud s
LoadRPMAvatar s
LockPatch b
MakeCircle s i f
MPLobby s
MPPopulatedWorld s s x
MPRandom
MPSession s
MPWorld s
MPWorldLast s
NextScene s
OpenURL s
OptoutAnalytics b
PlaceAllWindows
PlaceReflectionProbe
PlayModeNumbers b
PreloadWorld s
PreviousScene s
QuestBoostThreshold f
QuestUnBoostThreshold f
RemoveControlPanel
RGR
Script s
SelectAll
SendAnalyticsEvent s
SetAntiAliasing i
SetAudioEngineProcessingEnabled b
SetBufferSize i
SetColorPalette s
SetControllersVisible b
SetGameMode s
SetGhostRecordingGain f
SetGlobalSnapTurn b
SetGlobalSwimScaling b
SetGlobalTeleport b
SetGlobalUpsideDown b
SetGridEnabled b
SetMainLight f f f f f f
SetMarbleLifeTime f
SetMarbleMaxCount i
SetMarbleMaxDistance f
SetMetronomeEnabled b
SetMixedRealityMode s
SetPause b
SetPixelLightCount i
SetPlayerHeadPosition f f f f f f f
SetPlayerLimit f f f f f f
SetPlayerPosition f f f
SetPlayerScale f
SetPlayerScaleMultiplier f
SetPostProcessing i
SetPowerBlockVisible b
SetPrinterDefaultMode s
SetRadialMenuProfile i
SetRecordVoice b
SetRegion s
SetSceneBounciness f f f
SetShadowDistance f
SetShadowQuality i
SetSyncPhysicalSpace b
SetTempo f
SetTime f
SetWorldResolutionMultiplier f
SetWorldSnapTurn b
SetWorldSwimScaling b
SetWorldTeleport b
SetWorldUpsideDown b
ShowAllHidden b
ShowBlockList
ShowHidden b
ShowHub b
ShowTooltips b
SpawnAsync s
SwimToGazeAngle f
SwimToGazeWeight f
Test123
TestInvite
TestMP
TestRequest
TimelineLength f
TimelineLoop b
TimelinePlay
TimelineStop
ToPowerblock"""


##### Formation

In [ ]:

def split_by_capitals(text):
    result = re.findall(r'[A-Z][^A-Z]*', text)
    return ''.join(result) 


def split_by_new_line(text):
    # Use regex to split the text at capital letters
    # result = re.findall(r'[A-Z][^A-Z]*', text)
    result = text.split('\n')
    return result


def clean_and_split(text):
    # Remove newline characters
    text = text.replace('\n', ' ')
    
    # Split by the first space
    split_text = text.split(' ', 1)
    if len(split_text) > 1:
        split_text[1] = split_text[1].split()
    else:
        split_text.append('UNK')
    
    return split_text

def calc_embeddings(cmd, model):
    cmd_name = cmd[0]
    embedding = model.encode(cmd_name)
    cmd.append(embedding)

    return cmd



# cmds_sep = split_by_capitals(cmds_raw)
# cmds_with_args = [clean_and_split(c) for c in cmds_sep]
# cmd_with_embs = [calc_embeddings(c) for c in cmds_with_args]
# dict_with_methods = {cmd: [form, emb] for cmd, form, emb in cmd_with_embs}
# # list_with_methods = [[cmd, form, emb] for cmd, form, emb in cmd_with_embs]
# emb_to_method = {tuple(emb): [cmd, form] for cmd, form, emb in cmd_with_embs}
# # cmd_with_embs

In [ ]:
def get_cmd_emb(cmds_raw, model):
    cmds_sep = split_by_capitals(cmds_raw)
    cmds_with_args = [clean_and_split(c) for c in cmds_sep]
    cmd_with_embs = [calc_embeddings(c) for c in cmds_with_args]
    return cmd_with_embs


### Classification

In [ ]:
import psutil
import os
import time


def memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 ** 2  # in MB



In [20]:
from sklearn.metrics.pairwise import cosine_similarity


RED = '\033[31m'
RESET = '\033[0m'
YELLOW = '\033[33m'

intents = ["I want to change post processing", "I want to configure Autosaving setup", "Set tempo to 90 bmp", "Spawn new block"]

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# check_retrival(cmds_raw, model, intents)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [ ]:
def check_retrival(cmds_raw, model, intents, model_name='None'):
    num_params = sum([p.numel() for p in model.parameters()])
    print(70*'_')
    print(f"Model {YELLOW}{model_name}{RESET} with {YELLOW}{num_params / 1_000_000:.2f}M{RESET} of params")
    print(70*'-')
    print()

    # intent_embeddings = model.encode(intents)
    cmd_with_emb = get_cmd_emb(cmds_raw, model)
    # Measure start time and memory usage
    start_time = time.time()
    start_memory = memory_usage()

    embeddings_only = [emb for _, _, emb in cmd_with_emb]
    start_rec_time = time.time()


    for intent in intents:
        intent_emb = model.encode(intent).reshape(1, -1)
        similarities = cosine_similarity(intent_emb, embeddings_only)
        closest_intent_idx = similarities.argmax()
        predicted_intent = cmd_with_emb[closest_intent_idx][0]
        # print(f"for the intent {intent} found method {predicted_intent}.")
        print(f"for the intent {RED}{intent}{RESET} found method {RED}{predicted_intent}{RESET}.") 
    print()

    # Measure end time and memory usage
    end_time = time.time()
    end_memory = memory_usage()

    # Print performance metrics
    elapsed_time = end_time - start_time
    time_cycle = end_time - start_rec_time
    memory_consumed = end_memory - start_memory

    print(f'{elapsed_time=}')
    print(f'{time_cycle=}')
    print(f'{memory_consumed=}')


In [2]:

import os


# Verify it's set

In [ ]:
from openai import OpenAI
client = OpenAI()

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

def calc_embeddings_openai(cmd):
    cmd_name = cmd[0]
    embedding = get_embedding(cmd_name, model='text-embedding-3-small')
    cmd.append(embedding)
    return cmd

In [9]:
cmds_raw

'AbletonLinkEnabled b\nAbletonLinkLatency f\nAddRelatedWorld s\nAllowControlPanel b\nAllowEditing b\nAllowForking b\nAllowJoltPatching b\nAllowOpenBubbles b\nAllowPlayerScaling b\nAllowPlayerTilt b\nAllowRadialMenu b\nAllowSwimming b\nAmbientColorHSV f f f\nAmbientColorRGB f f f\nAssignAccessLevel i\nAutosaving b\nBGColor1 f f f\nBGColor2 f f f\nBgColorHSV f f f\nCallEvent i f\nChangeFogColor c\nChangeFogDistance f\nCheckChangeRate f\nCurrentStateToTemp\nDisableGhostHitInteractions b\nDisableGhostInteractions b\nDisplayButtonText b\nEditModeNumbers b\nEnableBeta b\nFadeToBlack f\nFogColorHSV f f f\nFogColorRGB f f f\nFogFalloffMode i\nGetCpuUsage\nGo s\nGoToTemp\nHideAll\nHighlightControllerButton s s f f\nLoadFromCloud s\nLoadRPMAvatar s\nLockPatch b\nMakeCircle s i f\nMPLobby s\nMPPopulatedWorld s s x\nMPRandom\nMPSession s\nMPWorld s\nMPWorldLast s\nNextScene s\nOpenURL s\nOptoutAnalytics b\nPlaceAllWindows\nPlaceReflectionProbe\nPlayModeNumbers b\nPreloadWorld s\nPreviousScene s\nQ

In [8]:
def get_cmd_emb_open(cmds_raw):
    cmds_sep = split_by_capitals(cmds_raw)
    cmds_with_args = [clean_and_split(c) for c in cmds_sep]
    cmd_with_embs = [calc_embeddings_openai(c) for c in cmds_with_args]
    return cmd_with_embs


In [ ]:
cmds_with_openai_emb = get_cmd_emb_open(cmds_raw)

In [ ]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a Liza assistant. Your name is public. You help with the game right now. Don't do respond anything else."},
        {
            "role": "user",
            "content": "Hi, who are you?"
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="I'm here to assist you with the game. How can I help you today?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
